# TP2 - DB Normalization and Querying

The objectives of this TP are:
1. Apply normalization 1NF -> 2NF -> 3NF
2. Perform SQL queries on the normalized database

In this TP, we will use a database **`wine.db`** (available in the course's website) containing wine information related to 'production' and 'sales'. 

> Production <---> Wine <---> Sales


---

### Working with db files in Jupyter
- Python provides an interface for SQLite through the *sqlite3* module
- The **`%%sql`** magic builds upon it (and other tools) to enable the usage of SQL commands within a Jupyter Notebook as in common SQL clients.
- Before proceeding, make sure that **`wine.db`** is in the same path as this notebook.
  - If **`wine.db`** is not in the same path, an empty **`wine.db`** file will be created, resulting in errors in later steps of the TP.
- The SQLite module in Python commits transactions automatically, this means that any change in the DB is immediately written to the file, e.g. creating/deleting tables.
  -  For this reason, it is recommended to have a backup of **`wine.db`** as it is provided in the course's website.

---

**`wine.db`** contains the following unnormalized tables:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

From wikipedia:

__Cru__: Often used to indicate a specifically named and legally defined vineyard or ensemble of vineyards and the vines "which grow on [such] a reputed terroir; by extension of good quality." The term is also used to refer to the wine produced from such vines.


<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [20]:
import sqlite3    # Python interface for SQLite databases

In [21]:
def printSchema(connection):
    # Function to print the DB schema
    # Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [22]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)           # An usefull way to viualize the content of the database

Database schema:
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
RBA:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
RBB:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
customer:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
  4: REGION(TEXT)
dummy:
  0: DEGRE(NUM)
producer:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
production:
  0: NV(NUM)
  1: NP(NUM)
  2: QTE(NUM)
sales:
  0: NV(NUM)
  1: NB(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: QTE(NUM)
wine:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)


From this point we will use __%%sql__ magic

In [3]:
%load_ext sql
%sql sqlite:///wine.db

'Connected: @wine.db'

# PART I: Database normalization

The first task on this TP is the normalization of the wine data. In its current state both tables **Master1** and **Master2** are in the First Normal Form (1NF).

By inspecting the content of these tables we can see that multiple tuples have NULL values.

In [6]:
%%sql SELECT *
FROM Master1
LIMIT 10;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
None,None,None,None,None,3,Six,Paul,Alsace
None,None,None,None,None,6,Marmagne,Bernard,Bourgogne
None,None,None,None,None,8,Lioger d'Harduy,Gabriel,Bourgogne
None,None,None,None,None,16,Barbin,Bernard,Bourgogne
None,None,None,None,None,17,Faiveley,Guy,Bourgogne
None,None,None,None,None,18,Tramier,Jean,Bourgogne
None,None,None,None,None,19,Dupaquier,Roger,Bourgogne
None,None,None,None,None,20,Lamy,Jean,Bourgogne
None,None,None,None,None,21,Cornu,Edmond,Bourgogne
None,None,None,None,None,26,Violot,Gilbert,Bourgogne


* Notice that Jupyter *displays* 'None' instead of 'NULL'. 
  - This is only to comply with python notation.
* To account for NULL values, your SQL queries must test explicitly for 'NULL'.

Another problem in **Master1** and **Master2** is data redundancy, for example:

In [7]:
%%sql SELECT *
FROM Master1
WHERE NV = 45;

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL,QTE,NP,NOM,PRENOM,REGION
45,Chiroubles,None,1983,90,2,Boxler,Albert,Alsace
45,Chiroubles,None,1983,912,67,Descombes,Jean Ernest,Beaujolais
45,Chiroubles,None,1983,98,71,Chalandard,Danile,Jura
45,Chiroubles,None,1983,540,78,Michlel,Pierre Emile,Jura
45,Chiroubles,None,1983,450,86,Dumazet,Marc,Rhone


---

Additional resource for Normalization:

https://www.youtube.com/watch?v=UrYLYV7WSHM

---

#### Exercise 1.1

Convert table **Master1** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.
  
Remember that **`wine.db`** contains information related to wine 'production' and 'sells'.

> Production <---> Wine <---> Sales

A good start point is to look for the 'Wine' attributes.

**Hint:** Look for redundant information between the master tables.

We can see from the preceding question that there is a redundancy between NV, CRU, DEGREE, MILL, QTE ...
We can consider the following functional dependencies :

NV --> CRU, DEGRE, MILL
NP --> NOM, PRENOM, REGION
NV, NP --> QTE

NV, NP is a key.
We can check this by : 

In [14]:
%%sql SELECT *
FROM (SELECT COUNT(*) AS count, NP, NV
FROM Master1
GROUP BY NP, NV) as en
ORDER BY en.count desc
LIMIT 5

 * sqlite:///wine.db
Done.


count,NP,NV
1,None,9
1,None,40
1,None,54
1,None,63
1,None,66


We can suggest a 3NF decomposition as:
WINE(NV,CRU,DEGREE,MILL,QTE), PRODUCER(NP,NOM,PRENOM,REGION) and PRODUCTION(NV,NP,QTE)

With:

table = PRODUCER | key = NP
table = WINE | key = NV
table = PRODUCTIONS | keys = (NV, NP)

#### Exercise 1.2

Convert table **Master2** to the Second Normal Form (2NF) or Third Normal Form (3NF) as applicable.
* Explain your answer
* List main functional dependencies (not all of them)
* Describe the schema of new tables and how they relate
  * You can write Tables as above or you can insert images in the notebook.

**Note:** For this part, consider that a wine can be bought in multiple locations and multiple times per day.

We can consider the following functional dependencies :

NB --> NOM, PRENOM, TYPE
NV --> CRU, DEGRE, MILL
NB, NV, Dates, Lieu --> QTE
LIEU --> REGION

NV, NB, Dates, LIEU is a key.
We can check this by : 

In [13]:
%%sql SELECT *
FROM
(SELECT COUNT(*) as count, NV, NB, DATES, LIEU
FROM Master2
GROUP BY NV,NB,DATES,LIEU) AS en
ORDER BY en.count DESC
LIMIT 5;

 * sqlite:///wine.db
Done.


count,NV,NB,DATES,LIEU
1,None,11,None,None
1,None,13,None,None
1,None,16,None,None
1,None,18,None,None
1,None,20,None,None


So a 3NF decomposition is:

WINE(NV,CRU,DEGREE,MILL), CUSTOMER(NB,NOM,PRENOM,TYPE), SALES(NV,NB,DATES,LIEU,QTE) and LOCATION(LIEU, REGION)


table = WINE | key = NV
table = CUSTOMER | key = NB
table = SALES | keys = (NV, NB, DATES, LIEU )
table = LOCATION | keys = LIEU 

Once you have defined the 2NF or 3NF (as applicable) we need to split the data into new tables.

A table can be created from the result of a query.

In the following example we will create a new table "dummy" to store the different values of alcohol content.

In [23]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1;

 * sqlite:///wine.db
Done.
Done.


[]

In [24]:
print("\nContent of the database")
printSchema(conn)


Content of the database
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
RBA:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
RBB:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
customer:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
  4: REGION(TEXT)
dummy:
  0: DEGRE(NUM)
producer:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
production:
  0: NV(NUM)
  1: NP(NUM)
  2: QTE(NUM)
sales:
  0: NV(NUM)
  1: NB(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: QTE(NUM)
wine:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)


In [25]:
%%sql
SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.


DEGRE
None
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8


Looking into "dummy", we notice that our query includes NULL. This is not allowed if we were to use DEGRE as key for a table.

To correct this, we need to change the query to explicitly test for NULL as follows:

In [26]:
%%sql DROP TABLE IF EXISTS dummy;

-- Create dummy table
CREATE TABLE dummy AS
SELECT DISTINCT DEGRE
FROM MASTER1
WHERE DEGRE IS NOT NULL;

SELECT *
FROM dummy;

 * sqlite:///wine.db
Done.
Done.
Done.


DEGRE
11.5
11.3
12.1
10.9
11.7
11.2
12.3
11.9
11.8
10.7


Notice that we use `NULL` given that `None` is only used for display.

In [27]:
# Remove "dummy" table
%sql DROP TABLE IF EXISTS dummy;

 * sqlite:///wine.db
Done.


[]

#### Exercise 1.3

Create the new tables from Master1:

In [28]:
%%sql 
DROP TABLE IF EXISTS producer;
DROP TABLE IF EXISTS wine;
DROP TABLE IF EXISTS production;

-- Create producer table
CREATE TABLE producer AS
SELECT DISTINCT NP,NOM,PRENOM,REGION
FROM MASTER1
WHERE NP IS NOT NULL;

-- Create wine table
CREATE TABLE wine AS
SELECT DISTINCT NV,CRU,DEGRE,MILL
FROM MASTER1
WHERE NV IS NOT NULL;

-- Create production table
CREATE TABLE production AS
SELECT DISTINCT NV,NP,QTE
FROM MASTER1
WHERE NP IS NOT NULL AND NV IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.
Done.
Done.
Done.
Done.


[]

#### Exercise 1.4

Create the new tables from Master2:

In [30]:
%%sql DROP TABLE IF EXISTS customer;
DROP TABLE IF EXISTS sales;
DROP TABLE IF EXISTS Location;

-- Create customer table
CREATE TABLE customer AS
SELECT DISTINCT NB,NOM,PRENOM,TYPE,REGION
FROM MASTER2
WHERE NB IS NOT NULL;

-- Create sales table
CREATE TABLE sales AS
SELECT DISTINCT NV,NB,DATES,LIEU,QTE
FROM MASTER2
WHERE NV IS NOT NULL AND NB IS NOT NULL;

-- Create Location table
CREATE TABLE Location AS
SELECT DISTINCT LIEU,Region
FROM MASTER2
WHERE LIEU IS NOT NULL AND REGION IS NOT NULL;

 * sqlite:///wine.db
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [34]:
print("\nContent of the database")
printSchema(conn)


Content of the database
Location:
  0: LIEU(TEXT)
  1: REGION(TEXT)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
RBA:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
RBB:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
customer:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
  4: REGION(TEXT)
producer:
  0: NP(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: REGION(TEXT)
production:
  0: NV(NUM)
  1: NP(NUM)
  2: QTE(NUM)
sales:
  0: NV(NUM)
  1: NB(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: QTE(NUM)
wine:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)


# PART II: SQL QUERIES

In the second part of this TP you will create SQL queries to retrieve information from the database.

**Important:**

- You MUST use the normalized tables created in previous steps.
  - The normalized tables will also be used in TP3.
- Do NOT use **Master1** and **Master2** in your queries.

#### Exercise 2.1

What are the different types of clients (buveurs) by volume of purchases?

In [32]:
%%sql

SELECT DISTINCT type
FROM customer

 * sqlite:///wine.db
Done.


TYPE
petit
moyen
gros


#### Exercise 2.2

What regions produce Pommard or Brouilly?

In [55]:
%%sql
SELECT DISTINCT region 
FROM wine w
INNER JOIN production pr ON w.NV = pr.NV
INNER JOIN producer p ON p.NP = pr.NP
WHERE w.CRU = 'Pommard' OR w.CRU = 'Brouilly'

 * sqlite:///wine.db
Done.


REGION
Bourgogne
Rhone


#### Exercise 2.3

What regions produce Pommard and Brouilly?

In [56]:
%%sql
SELECT DISTINCT region 
FROM wine w
INNER JOIN production pr ON w.NV = pr.NV
INNER JOIN producer p ON p.NP = pr.NP
WHERE w.CRU = 'Pommard' OR w.CRU = 'Brouilly'

 * sqlite:///wine.db
Done.


REGION
Bourgogne
Rhone


#### Exercise 2.4

Get the number of wines bought by CRU and Millésime

In [40]:
%%sql
SELECT DISTINCT COUNT(*) as count, CRU, MILL
FROM wine 
GROUP BY CRU, MILL 
ORDER BY count DESC

 * sqlite:///wine.db
Done.


count,CRU,MILL
2,Chiroubles,1983
2,Cornas,1980
2,Cotes du Rousillon,1978
2,Mercurey,1980
2,Saint Chinian,1976
2,Savigny les Beaunes,1978
2,Seyssel,1980
2,Sylvaner,1982
2,Tavel,1980
1,Arbois,1976


#### Exercise 2.5

Retrieve the wine number (NV) of wines produced by more than three producers

In [57]:
%%sql
SELECT DISTINCT COUNT(*) as count, w.NV
FROM wine w
INNER JOIN production pr ON w.NV = pr.NV
GROUP BY w.NV 
HAVING count >= 3

 * sqlite:///wine.db
Done.


count,NV
3,12
3,43
5,45
3,54
3,67
3,69
3,74
5,78
3,79
3,87


#### Exercise 2.6

Which producers have not produced any wine?

In [58]:
%%sql
SELECT *
FROM producer
WHERE NP NOT IN (
    SELECT NP
    FROM production)

 * sqlite:///wine.db
Done.


NP,NOM,PRENOM,REGION
3,Six,Paul,Alsace
6,Marmagne,Bernard,Bourgogne
8,Lioger d'Harduy,Gabriel,Bourgogne
16,Barbin,Bernard,Bourgogne
17,Faiveley,Guy,Bourgogne
18,Tramier,Jean,Bourgogne
19,Dupaquier,Roger,Bourgogne
20,Lamy,Jean,Bourgogne
21,Cornu,Edmond,Bourgogne
26,Violot,Gilbert,Bourgogne


#### Exercise 2.7

What clients (buveurs) have bought at least one wine from 1980?

In [69]:
%%sql
SELECT  DISTINCT c.* , w.MILL
FROM customer c
INNER JOIN sales s ON c.NB = s.NB
INNER JOIN wine w ON w.NV = s.NV
WHERE c.NB IN
(
    SELECT s.NB
    FROM customer c2 INNER JOIN sales s ON c2.NB = s.NB
    INNER JOIN wine w ON w.NV = s.NV
    WHERE w.MILL = 1980
)

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE,MILL
2,Artaud,Antonin,moyen,1980
2,Artaud,Antonin,moyen,1976
2,Artaud,Antonin,moyen,1979
44,Gide,Andre,petit,1980
45,Giono,Jean,petit,1980
8,Aragon,Louis,gros,1980
8,Aragon,Louis,gros,1976
50,Lautreamont,None,moyen,1980
61,Mallarme,Stephane,petit,1973
61,Mallarme,Stephane,petit,1980


#### Exercise 2.8

What clients (buveurs) have NOT bought any wine from 1980?

In [66]:
%%sql
SELECT DISTINCT c.*, w.MILL
FROM customer c
INNER JOIN sales s ON c.NB = s.NB
INNER JOIN wine w ON w.NV = s.NV
WHERE c.NB NOT IN
(
    SELECT c.NB
    FROM customer c
    INNER JOIN sales s ON c.NB = s.NB
    INNER JOIN wine w ON w.NV = s.NV
    WHERE w.MILL = 1980
)
ORDER BY c.NB

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE,MILL
1,Aristote,None,petit,1975
3,Aron,Raymond,gros,1981
3,Aron,Raymond,gros,1979
3,Aron,Raymond,gros,1978
4,Apollinaire,Guillaume,moyen,1968
5,Audiberti,Jacques,petit,1973
5,Audiberti,Jacques,petit,1979
5,Audiberti,Jacques,petit,1974
6,Arrabal,Fernando,gros,1975
7,Anouilh,Jean,moyen,1978


#### Exercise 2.9

What clients (buveurs) have bought ONLY wines from 1980?

In [74]:
%%sql
SELECT DISTINCT c.*, w.MILL
FROM customer c
INNER JOIN sales s ON c.NB = s.NB
INNER JOIN wine w ON w.NV = s.NV
WHERE c.NB NOT IN
(
    SELECT c.NB
    FROM customer c
    INNER JOIN sales s ON c.NB = s.NB
    INNER JOIN wine w ON w.NV = s.NV
    WHERE w.MILL <> 1980
)
ORDER BY c.NB

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE,MILL
44,Gide,Andre,petit,1980
45,Giono,Jean,petit,1980
50,Lautreamont,None,moyen,1980


#### Exercise 2.10

List all wines from 1980

In [73]:
%%sql
SELECT *
FROM wine
WHERE MILL = 1980

 * sqlite:///wine.db
Done.


NV,CRU,DEGRE,MILL
1,Mercurey,11.5,1980
4,Mercurey,10.9,1980
16,Meursault,12.1,1980
20,Cote de Brouilly,12.1,1980
26,Chateau Corton Grancey,None,1980
28,Volnay,11,1980
43,Fleurie,11.4,1980
74,Arbois,12,1980
78,Etoile,12,1980
79,Seyssel,11,1980


#### Exercise 2.11

What are the wines from 1980 bought by NB=2?

In [77]:
%%sql
SELECT w.NV, CRU, MILL
FROM wine w
INNER JOIN sales s ON w.NV = s.NV
WHERE MILL = 1980 AND s.NB = 2

 * sqlite:///wine.db
Done.


NV,CRU,MILL
1,Mercurey,1980


#### Exercise 2.12

What clients (buveurs) have bought ALL the wines from 1980?

In [84]:
%%sql
SELECT *
FROM customer c
    INNER JOIN sales s ON c.NB = s.NB
    INNER JOIN Wine w on s.NV = w.NV
WHERE w.MILL =1980
GROUP BY c.NB
HAVING COUNT(*) = (SELECT COUNT(DISTINCT w.NV) FROM Wine w WHERE w.MILL=1980)

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE,NV,NB_1,DATES,QTE,NV_1,CRU,DEGRE,MILL
44,Gide,Andre,petit,87,44,2015-10-16,18,87,Cotes de Provence,None,1980
